In [1]:
import os
import json
import glob
import dateutil.parser as dparser

import imageio
import numpy as np
import moviepy.editor as mp

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pgf import FigureCanvasPgf

mpl.backend_bases.register_backend("pdf", FigureCanvasPgf)
mpl.use("pgf")
%matplotlib inline
%config InlineBackend.figure_format="retina"

# Matplotlib plotting options
plt.style.use("seaborn-paper")
plt.rcParams.update({
    "axes.labelsize": 16,               # label size (x- and y-axis)
    "axes.titlesize": 20,               # title size on (most) axes
    "figure.titlesize": 20,             # title size for fig.sup_title()
    "legend.fontsize": "large",         # font size for legend
    "lines.markersize": 6,              # marker size for points and lines
    "lines.markeredgewidth": 2,         # marker edgewidth for points
    "xtick.labelsize": 14,              # label size for x-ticks 
    "ytick.labelsize": 14,              # label size for y-ticks

    "font.family": "serif",             # use serif/main font for text elements
    "text.usetex": True,                # use inline math for ticks
    "pgf.rcfonts": False,               # don't setup fonts from rc params
    "pgf.preamble": [
        # Syling
        r"\usepackage{color}",          # special colors
        r"\setmainfont{DejaVu Serif}",  # serif font via preamble

        # Math
        r"\usepackage{xfrac}",          # side fractions
        r"\usepackage{amsthm}",         # theorems
        r"\usepackage{amsmath}",        # misc math
        r"\usepackage{amssymb}",        # blackboard math symbols
        r"\usepackage{mathtools}",      # enhance the appearance of math
    ],
})

In [2]:
def corner(data):
    """Corner plot of latent distribution :math:`z`."""
    num_latent = data.shape[-1]
    figsize = (num_latent*3, num_latent*3)
    fig, axarr = plt.subplots(nrows=num_latent, ncols=num_latent, sharex=True,
                              sharey=False, figsize=figsize)
    for i in range(axarr.shape[0]):
        # Plot latent vector (z) for each param
        for j in range(i+1):
            ax = axarr[i,j]
            if i == j:
                ax.hist(data[:, j], bins=20, color="mediumseagreen", density=True)
            else:
                ax.scatter(data[:, i], data[:, j],  c="mediumseagreen", marker="o",
                           alpha=1.0, edgecolors="black")
                ax.set_xlim([-5, 5])
                ax.set_ylim([-5, 5])
        # Make unused plots white
        for j in range(i+1, axarr.shape[1]):
            axarr[i,j].axis('off')

    # Cleanup
    for i, row in enumerate(axarr):
        for j, cell in enumerate(row):
            # Display labels on edge plots only
            if i == len(axarr) - 1:
                cell.set_xlabel("$q(z_{%d}|x)$" % j)
            if j == 0:
                cell.set_ylabel("$q(z_{%d}|x)$" % i)
            # Remove tick labels on inner subplots
            if i != len(axarr) - 1:
                cell.set_xticklabels([])
            if j != 0:
                cell.set_yticklabels([])
    return fig

In [3]:
epoch = 0
rootdir = "../../results/"
files = sorted(glob.glob("../../dumps/3gb1/vae/*/*.json"))
for file in files:
    # Epoch number
    epoch += 1
    # Parse datetime from the file
    for _str in file.split("/"):
        try:
            dt = dparser.parse(_str, fuzzy=True)
        except ValueError:
            pass
    ts = dt.strftime("%Y-%b-%d-%H:%M:%S")
    savedir = os.path.join(rootdir, f"3gb1/vae/{ts}")
    os.makedirs(savedir, exist_ok=True)

    with open(file, "r") as fp:
        dump = json.load(fp)
    # Obtain original (encoded) data and latent (z) space
    z = np.array(dump["z"])
    fig = corner(z[:, :5])
    # Cleanup and save fig
    fig.tight_layout()
    fig.suptitle(f"Latent vector (epoch {epoch:04d})")
    fig.subplots_adjust(top=0.95)
    fig.savefig(os.path.join(savedir, f"E{epoch:04d}.png"),
                bbox_inches="tight", dpi=150)
    plt.close()

In [4]:
filenames = sorted(glob.glob(os.path.join(savedir, "E*.png")))
with imageio.get_writer(os.path.join(savedir, 'latent.gif'), mode='I', duration=0.2) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

clip = mp.VideoFileClip(os.path.join(savedir, 'latent.gif'))
clip.write_videofile(os.path.join(savedir, 'latent.mp4'))

t:   0%|          | 0/10 [00:00<?, ?it/s, now=None]

Moviepy - Building video ../../dumps/3gb1/vae/2020-Apr-27-20:15:46/latent.mp4.
Moviepy - Writing video ../../dumps/3gb1/vae/2020-Apr-27-20:15:46/latent.mp4



Moviepy - Done !
Moviepy - video ready ../../dumps/3gb1/vae/2020-Apr-27-20:15:46/latent.mp4
